#### - Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
#### - Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
#### - Graficarlos.
#### - Obtener conclusiones.

In [62]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np   
import plotly.graph_objects as go
import plotly.express as px                               

from tqdm import tqdm
from gensim.models.callbacks import CallbackAny2Vec


import multiprocessing
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import plotly.io as pio

pio.renderers.default = 'browser'
plt.style.use('dark_background')

Para el desafío, usaremos una colección de los 25 libros más descargados de la web de proyecto Gutenberg. Teniendo una lista de ids de libros obtenidas por curl, iteremos sobre la misma para descargar los libros:

In [26]:

book_ids = pd.read_csv("data/d2/books_ids.csv")
total_books = len(book_ids)

for book_id in tqdm(book_ids['book_id'], desc="Descargando libros. . .", total=total_books):
    url = f"https://www.gutenberg.org/ebooks/{book_id}.txt.utf-8"
    output_dir= "data/d2/books"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Lanza un error si la descarga falla
        with open(os.path.join(output_dir, f"{book_id}.txt"), 'w', encoding='utf-8') as file:
            file.write(response.text)
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar el libro {book_id}: {e}")

Descargando libros. . .:  76%|███████▌  | 19/25 [00:37<00:10,  1.73s/it]

Error al descargar el libro 38769: 404 Client Error: Not Found for url: https://www.gutenberg.org/cache/epub/38769/pg38769.txt


Descargando libros. . .: 100%|██████████| 25/25 [00:48<00:00,  1.95s/it]


Leemos cada uno de los .txt y segmentamos:

In [29]:
dfs = []
for file in os.listdir(output_dir):
    if file.endswith('.txt'):
        dfx = pd.read_csv(os.path.join(output_dir, file), sep='/n', header=None, names=['Sentence'], engine='python')
        dfs.append(dfx)
df = pd.concat(dfs, ignore_index=True)
print(df.shape[0])
df.head()

447562


,Sentence
0,The Project Gutenberg eBook of The Complete Wo...
1,This ebook is for the use of anyone anywhere i...
2,most other parts of the world at no cost and w...
3,"whatsoever. You may copy it, give it away or r..."
4,of the Project Gutenberg License included with...


Preprocesamos el dataset como vimos en clase

In [33]:
sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df.iloc[:].iterrows():
    sentence_tokens.append(text_to_word_sequence(row.iloc[0]))

Realizamos nuestra vectorización

In [34]:
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [35]:
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [36]:
w2v_model.build_vocab(sentence_tokens)
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de docs en el corpus: 447562
Cantidad de words distintas en el corpus: 24699


Entrenamos el embedding:

In [38]:
for epoch in tqdm(range(20), desc="Entrenando . . .", total=20):
    w2v_model.train(sentence_tokens,
                     total_examples=w2v_model.corpus_count,
                     epochs=1,
                     compute_loss=True,
                     callbacks=[callback()]
                     )
   

Entrenando . . .:   5%|▌         | 1/20 [00:37<12:01, 37.96s/it]

Loss after epoch 0: 20991722.0


Entrenando . . .:  10%|█         | 2/20 [01:16<11:25, 38.11s/it]

Loss after epoch 0: 20474404.0


Entrenando . . .:  15%|█▌        | 3/20 [01:55<10:58, 38.71s/it]

...

Loss after epoch 0: 18947492.0


Entrenando . . .:  95%|█████████▌| 19/20 [12:53<00:43, 43.57s/it]

Loss after epoch 0: 18912818.0


Entrenando . . .: 100%|██████████| 20/20 [13:34<00:00, 40.70s/it]

Loss after epoch 0: 18853790.0


Analizamos similitudes en espacio de embedding como vimos en clase, comparando las caracteristicas de cada término en coseno similitud:

In [41]:
w2v_model.wv.most_similar(positive=["father"], topn=10)


[('mother', 0.5008445382118225),
 ('clerval', 0.4751327335834503),
 ('stepmother', 0.4726009666919708),
 ('lucie', 0.4715188443660736),
 ('ratchcali', 0.4673488438129425),
 ('ottilie', 0.46160653233528137),
 ('justine', 0.46006518602371216),
 ('espousing', 0.45555177330970764),
 ('rosalie', 0.45282384753227234),
 ('gordons', 0.4520285129547119)]

En el caso de "father" podemos ver palabras que son similares ya que representan un vínculo familair ascendente, y otros términos relativos a los libros con que entrenamos (Nombres de personajes que son padres y madres, que en el corpus se usan como términos similares).

In [56]:
w2v_model.wv.most_similar(positive=["money"], topn=10)


[('forborne', 0.46100419759750366),
 ('luggage', 0.45130211114883423),
 ('cash', 0.4449760913848877),
 ('funds', 0.4422464668750763),
 ('premium', 0.43783068656921387),
 ('herewith', 0.4362329840660095),
 ('dollar', 0.4298473000526428),
 ('invoice', 0.4291066825389862),
 ('eighty', 0.4268193542957306),
 ('quarts', 0.4251228868961334)]

Visualizamos la cercanía de los terminos reduciendo la dimensionalidad de embedding con PCA, con el fin de poder acomodarlos en gráficas:

In [64]:
def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

vecs, labels = reduce_dimensions(w2v_model, 3)

Observamos un gráfico tridimensional que posiciona los términos basándose en las tres características de PCA:

_(Nota: el gráfico se abrirá en navegador web)._

In [65]:

MAX_WORDS = 200  
fig = px.scatter_3d(x=vecs[:MAX_WORDS, 0], y=vecs[:MAX_WORDS, 1], z=vecs[:MAX_WORDS, 2], text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)